#Load data

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dialogues_018.json to dialogues_018.json


In [2]:
# intents
!mkdir -p dialogue_018/NONE
!mkdir -p dialogue_018/SearchOnewayFlight
!mkdir -p dialogue_018/ReserveOnewayFlight
!mkdir -p dialogue_018/SearchRoundtripFlights
!mkdir -p dialogue_018/ReserveRoundtripFlights

Script para coletar apenas as utterances (frases) do usuário e armazená-las num diretório que representa a intenção(que será nossa classe para a tarefa de classificação de intenção).</br>
<b>atenção</b>: O sistema foi excluído da coleta de utterances porque a resposta do sistema não possui intenção

In [3]:
import json

if __name__ == '__main__':
    with open("dialogues_018.json", 'r') as file:
        data_018 = json.load(file)

    for i in range(len(data_018)):
        frames_amount = len(data_018[i]['turns'])
        for j in range(frames_amount):
            current_frame = data_018[i]['turns'][j]
            speaker = current_frame['speaker']
            if speaker == 'USER':
                intent = current_frame['frames'][0]['state']['active_intent']
                utterance = current_frame['utterance']

                file_path = "dialogue_018/" + intent + "/" + str(i) + "_" + str(j) + ".txt"

                f = open(file_path, "w+")
                f.write(utterance)
                f.close()


Armazenar numa estrutura para treinamento no Keras

In [4]:
import tensorflow as tf

batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    '/content/dialogue_018',
    labels='inferred',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 1052 files belonging to 5 classes.
Using 842 files for training.


# Training

https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU <br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

**PRIMEIRA PARTE**
</br>
Classificador para classificação de domínio, intenção e extração dos campos;
</br>
Requer dados rotulados;

In [ ]:
data[0]['turns'][0]['frames'][0]['state']

{'active_intent': 'SearchOnewayFlight',
 'requested_slots': [],
 'slot_values': {'seating_class': ['Premium Economy']}}

In [ ]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GRU
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Extract dialogues and their labels
dialogues = []
labels = []

for dialogue in data:
  for turn in dialogue['turns']:
    for frame in turn['frames']:
      if turn['speaker'] == 'SYSTEM':
          service_name = frame['service']
          labels.append(service_name)
      elif turn['speaker'] == 'USER':
        active_intent = frame['state']['active_intent']
        labels.append(active_intent)

    dialogues.append(turn['utterance'])

print(len(dialogues))
print(len(labels))

# Tokenize and pad sequences
max_words = 10000
max_sequence_length = 100
embedding_dim = 50

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dialogues)
sequences = tokenizer.texts_to_sequences(dialogues)

X = np.array(sequences)

# Convert labels to numerical representation using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# Convert to one-hot encoding
y_one_hot = to_categorical(y_encoded, num_classes=4)

# Define the LSTM model
#model = Sequential()
#model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length))
#model.add(LSTM(100))
#model.add(Dense(4, activation='softmax'))

# Define the GRU model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_sequence_length))
model.add(GRU(100))  # Replace LSTM with GRU
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y_one_hot, epochs=5, validation_split=0.2)

# Save the model
model.save('dialogue_classification_model.h5')


### Treinamento com BERT

In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig()  # default parameters and configuration for BERT


In [ ]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Testing

Podemos fazer uma interface pelo terminal

# Evaluation